In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import vk_api
import os
import requests
import json
import random
import sys

In [2]:
sns.set(
    font_scale =2,
    style      ='whitegrid',
    rc         ={'figure.figsize':(20,7)})

## VK API

In [2]:
token = 'c04c52b2dd5cfba94a85c388951c663230a5de49e0cefaced73a0fcf2770ac8cf0eb41a29f55b2b9dbd4f'

In [9]:
vk_session = vk_api.VkApi(token=token)
vk= vk_session.get_api()

In [12]:
vk.messages.send(
    chat_id=1,
    random_id=2,
    message='Matrix has you...')

0

In [21]:
df = pd.read_csv('ads_data.csv')
df.columns

Index(['date', 'time', 'event', 'platform', 'ad_id', 'client_union_id',
       'campaign_union_id', 'ad_cost_type', 'ad_cost', 'has_video',
       'target_audience_count'],
      dtype='object')

In [24]:
ad_data = df.groupby(['ad_id','event'],as_index=False) \
.agg({'campaign_union_id': 'count'})

In [27]:
ad_data = ad_data.pivot(index='ad_id', 
              columns='event',
              values='campaign_union_id').reset_index()

In [28]:
ad_data = ad_data.fillna(0).assign(ctr= ad_data.click / ad_data.view)

In [39]:
top_ctr = ad_data.query('click > 20 and view > 100') \
.sort_values('ctr').tail(10)

In [40]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

## Отправка файла в лс

In [41]:
path = 'c:top_ctr_data.csv'

In [43]:
file_name = 'top_ctr_data.csv'

In [42]:
path_to_file = path

In [58]:
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)['upload_url']

In [59]:
file = {'file': (file_name, open(path_to_file, 'rb'))}

In [60]:
response = requests.post(upload_url, files=file)

In [61]:
json_data = json.loads(response.text)

In [62]:
json_data

{'file': '40431466|0|-1|536236|b64754a725|csv|411|top_ctr_data.csv|c1ba3c40c16b2b4a0a3d599d97c7661c|2a768d57d6497828883b1da7786a4653||||eyJkaXNrIjoiMTIiLCJwZWVyX3NlbmRlciI6Ii0yMDQxMDg4MjkifQ=='}

In [66]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [67]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

In [68]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='Топ объявлений по CTR',
    attachment = attachment
)

0

## Google doc API

In [6]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

## API Яндекс метрики